# TF Data Preparation

In [2]:
from PetReader import pet_reader
import tensorflow as tf
import tensorflow_addons  as tfa
import transformers
from petreader.labels import *

Reusing dataset pet (C:\Users\janek\.cache\huggingface\datasets\patriziobellan___pet\relations-extraction\1.0.1\38434e2af57af533c400c8975f37e43c08bb77739085a3c026a862b2efb668d2)




 _______ _     _ _______       _____  _______ _______      ______  _______ _______ _______ _______ _______ _______
    |    |_____| |______      |_____] |______    |         |     \ |_____|    |    |_____| |______ |______    |   
    |    |     | |______      |       |______    |         |_____/ |     |    |    |     | ______| |______    |   
                                                                                                                  
Discover more at: [https://pdi.fbk.eu/pet-dataset/]



  0%|          | 0/1 [00:00<?, ?it/s]

Reusing dataset pet (C:\Users\janek\.cache\huggingface\datasets\patriziobellan___pet\token-classification\1.0.1\38434e2af57af533c400c8975f37e43c08bb77739085a3c026a862b2efb668d2)




 _______ _     _ _______       _____  _______ _______      ______  _______ _______ _______ _______ _______ _______
    |    |_____| |______      |_____] |______    |         |     \ |_____|    |    |_____| |______ |______    |   
    |    |     | |______      |       |______    |         |_____/ |     |    |    |     | ______| |______    |   
                                                                                                                  
Discover more at: [https://pdi.fbk.eu/pet-dataset/]



  0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
import argparse

## Create Dataset

In [3]:
tokenizer = transformers.DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

In [4]:
sample_numbers = pet_reader.token_dataset.GetRandomizedSampleNumbers()
sample_dicts = [pet_reader.token_dataset.GetSampleDictWithNerLabels(sample_number) for sample_number in sample_numbers]
sample_sentences = [sample_dict['tokens'] for sample_dict in sample_dicts]

# 1) transform tokens tags into IDs classification
all_tokens = tokenizer(sample_sentences, is_split_into_words=True, padding=True, return_tensors='tf')
max_sentence_length = all_tokens['input_ids'].shape[1]

# 2) transform NER token tags into labels for classification
all_sample_labels = []
for i, sample_number in enumerate(sample_numbers):
    sample_dict = pet_reader.token_dataset.GetSampleDictWithNerLabels(sample_number)
    # transformer_tokens = tokenizer.convert_ids_to_tokens(tokens['input_ids'][i])
    # tokenize again every single sample to get access to .word_ids()
    tokenization = tokenizer(sample_dict['tokens'], is_split_into_words=True, 
                             padding='max_length', max_length=max_sentence_length, return_tensors='tf')
    sample_tokens = tokenizer.convert_ids_to_tokens(tokenization['input_ids'][0])
    
    sample_labels = []
    # word index necessary, because one token in PET could be splitted into multiple tokens with tokenizer
    # multiple tokens have all the same word_id -> allows retrieval of the same one NER label from PET tokens
    for token, word_index in zip(sample_tokens, tokenization.word_ids()):
        # set special class for special tokens
        if token in ['[CLS]', '[SEP]', '[PAD]']:
            sample_labels.append(0)
        else:
            token_tag = sample_dict['ner-tags'][word_index]
            if token_tag.endswith(XOR_GATEWAY):
                sample_labels.append(2)
            elif token_tag.endswith(AND_GATEWAY):
                sample_labels.append(3)
            else:
                sample_labels.append(1)
                
    all_sample_labels.append(sample_labels)

# print(all_sample_labels)
labels = tf.constant(all_sample_labels)
tokens = all_tokens
print(labels.shape)
print(tokens['input_ids'].shape)

(417, 64)
(417, 64)


In [5]:
print(labels.shape)
print(tokens['input_ids'].shape)

dataset = tf.data.Dataset.from_tensor_slices(({'input_ids': tokens['input_ids'], 'attention_mask': tokens['input_ids']}, 
                                              labels))
# dataset = dataset.batch(8)

(417, 64)
(417, 64)


In [6]:
# split up
val_share = 0.1
val_instances = round(tokens['input_ids'].shape[0] * val_share)
dev_dataset = dataset.take(val_instances) 
train_dataset = dataset.skip(val_instances) 
print(f"{len(train_dataset)} / {len(dev_dataset)}")

# batch both
dev_dataset = dev_dataset.batch(8)
train_dataset = train_dataset.batch(8)
print(f"Batched: {len(train_dataset)} / {len(dev_dataset)}")

375 / 42
Batched: 47 / 6


## Create Model

In [7]:
token_cls_model = transformers.TFAutoModelForTokenClassification.from_pretrained("distilbert-base-uncased", num_labels=4)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForTokenClassification: ['vocab_transform', 'vocab_layer_norm', 'vocab_projector', 'activation_13']
- This IS expected if you are initializing TFDistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inferenc

In [12]:
type(token_cls_model)

transformers.models.distilbert.modeling_tf_distilbert.TFDistilBertForTokenClassification

### a) including Model in native Keras

In [11]:
# setting up training parameters
batch_size = 8
num_train_epochs = 3
num_train_steps = (len(train_dataset) // 8) * num_train_epochs
optimizer, lr_schedule = transformers.create_optimizer(
    init_lr=2e-5,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
    num_warmup_steps=0,
)

# compile model
token_cls_model.compile(optimizer=optimizer,
                        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                        metrics=[tf.metrics.SparseCategoricalAccuracy(name="accuracy"),
#                                  tf.metrics.Precision(name="precision"),
#                                  tf.metrics.Recall(name="recall"),
#                                  tfa.metrics.F1Score(4, name="f1")
                                ])
token_cls_model.summary()

# train the model
token_cls_model.fit(train_dataset, 
                      validation_data=dev_dataset, 
                      epochs=1)

Model: "tf_distil_bert_for_token_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 dropout_19 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  3076      
                                                                 
Total params: 66,365,956
Trainable params: 66,365,956
Non-trainable params: 0
_________________________________________________________________


ValueError: in user code:

    File "C:\Users\janek\anaconda3\envs\thesis\lib\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\janek\anaconda3\envs\thesis\lib\site-packages\keras\engine\training.py", line 1010, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\janek\anaconda3\envs\thesis\lib\site-packages\keras\engine\training.py", line 1000, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\janek\anaconda3\envs\thesis\lib\site-packages\transformers\modeling_tf_utils.py", line 1156, in train_step
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    File "C:\Users\janek\anaconda3\envs\thesis\lib\site-packages\keras\engine\compile_utils.py", line 459, in update_state
        metric_obj.update_state(y_t, y_p, sample_weight=mask)
    File "C:\Users\janek\anaconda3\envs\thesis\lib\site-packages\ker

### b) with transformers Trainer (NOT WORKING WITH TF)

In [13]:
batch_size = 8
args = transformers.TrainingArguments(
    output_dir='./results',
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01
)

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [p for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [l for (p, l) in zip(prediction, label) if l != -100] 
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

trainer = transformers.TFTrainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

C:\Users\janek\anaconda3\envs\thesis\lib\site-packages\transformers\trainer_tf.py:115: FutureWarning: The class `TFTrainer` is deprecated and will be removed in version 5 of Transformers. We recommend using native Keras instead, by calling methods like `fit()` and `predict()` directly on the model object. Detailed examples of the Keras style can be found in our examples at https://github.com/huggingface/transformers/tree/main/examples/tensorflow
  warnings.warn(


ImportError: Method `n_gpu` requires PyTorch.